In [9]:
import glob
import os
import re
import subprocess
import shutil
import yaml
from pathlib import Path

In [10]:
m = yaml.safe_load(open('/data/pecgs/test_sample/C3L-00677/summary.yaml'))
m

{'id': 'C3L-00677',
 'rna-seq': {'tumor': {'R1': {'filepath': '/data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
   'R2': {'filepath': '/data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}}},
 'wxs': {'normal': {'barcode': 'CPT0071960002',
   'filepath': '/data/pecgs/test_sample/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
  'tumor': {'barcode': 'CPT0071100006',
   'filepath': '/data/pecgs/test_sample/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}}

###### top

In [14]:
GENOME = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'
GENOME_DICT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'
TARGET_INTERVAL_LIST = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list'
COMMON_BIALLELIC = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
PROTEIN_CODING_GENE = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed'
POOL_OF_NORMALS = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5'


###### step 3: STAR

In [8]:
# # sh $submitJob 16 1 gatk4cn.s3.${caseID} "bash $scriptDir/src/3.run.gatk4scna.callCNA.pair.sh -C ${config} -P ${CNVPON} -S ${caseID} -N ${normalBam} -T ${tumorBam} -O ${outdir}"


# ##---------- TotalCNA Tumor BAM
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} CollectReadCounts \
#       -I ${BAM_Tumor} \
#       -L ${TargetIntervalList} \
#       --interval-merging-rule OVERLAPPING_ONLY \
#       -O ${OUT}/${caseName}.T.counts.hdf5



# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} DenoiseReadCounts \
#       -I ${OUT}/${caseName}.T.counts.hdf5 \
#       --count-panel-of-normals ${CNVPON} \
#       --standardized-copy-ratios ${OUT}/${caseName}.T.standardizedCR.tsv \
#       --denoised-copy-ratios ${OUT}/${caseName}.T.denoisedCR.tsv


# # Need large memory
# ##---------- AlleleCNA Normal BAMs
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} CollectAllelicCounts \
#       -L ${COMMON_BIALLELIC} \
#       -I ${BAM_Normal} \
#       -R ${GENOME} \
#       -O ${OUT}/${caseName}.N.allelicCounts.tsv



# ##---------- AlleleCNA Tumor BAMs
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} CollectAllelicCounts \
#       -L ${COMMON_BIALLELIC} \
#       -I ${BAM_Tumor} \
#       -R ${GENOME} \
#       -O ${OUT}/${caseName}.T.allelicCounts.tsv

      

# ##---------- AlleleCNA Nor & Tum 
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} ModelSegments \
#       --denoised-copy-ratios ${OUT}/${caseName}.T.denoisedCR.tsv \
#       --allelic-counts ${OUT}/${caseName}.T.allelicCounts.tsv \
#       --normal-allelic-counts ${OUT}/${caseName}.N.allelicCounts.tsv \
#       --output ${OUT} \
#       --output-prefix ${caseName}.T
    

    
# ##---------- Segment 
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx16g -jar ${GATK} CallCopyRatioSegments \
#       --input ${OUT}/${caseName}.T.cr.seg \
#       --output ${OUT}/${caseName}.T.called.seg

In [13]:
# ##---------- Plot-TotalCN
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx8g -jar ${GATK} PlotDenoisedCopyRatios \
#       --standardized-copy-ratios ${OUT}/${NAME}.standardizedCR.tsv \
#       --denoised-copy-ratios ${OUT}/${NAME}.denoisedCR.tsv \
#       --sequence-dictionary ${GENOME_DICT} \
#       --minimum-contig-length ${min_contigLen} \
#       --output ${OUT}/plots \
#       --output-prefix ${NAME} \
#       --tmp-dir ${OUT}



# ##---------- Plot-alleleCN
# ${JAVA}  -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -jar ${GATK} PlotModeledSegments \
#       --denoised-copy-ratios ${OUT}/${caseName}.T.denoisedCR.tsv \
#       --allelic-counts ${OUT}/${caseName}.T.hets.tsv \
#       --segments ${OUT}/${caseName}.T.modelFinal.seg \
#       --sequence-dictionary ${GENOME_DICT} \
#       --minimum-contig-length ${min_contigLen} \
#       --output ${OUT}/plots \
#       --output-prefix ${caseName}.T \
#       --tmp-dir ${OUT}

In [15]:
def collect_read_counts(tumor_bam, sample, out_dir):
    pieces = [
        'gatk CollectReadCounts',
        '-I', tumor_bam,
        '-L', TARGET_INTERVAL_LIST,
        '--interval-merging-rule OVERLAPPING_ONLY',
        '-O', os.path.join(out_dir, f'{sample}.T.counts.hdf5')
    ]
    return ' '.join(pieces)


def denoise_counts(sample, out_dir):
    pieces = [
        'gatk DenoiseReadCounts',
        '-I', os.path.join(out_dir, f'{sample}.T.counts.hdf5'),
        '--count-panel-of-normals', POOL_OF_NORMALS,
        '--standardized-copy-ratios', os.path.join(out_dir, f'{sample}.T.standardizedCR.tsv'),
        '--denoised-copy-ratios', os.path.join(out_dir, f'{sample}.T.denoisedCR.tsv')
    ]
    return ' '.join(pieces)


def collect_allelic_counts(bam, sample, out_dir, char):
    pieces = [
        'gatk CollectAllelicCounts',
        '-L', COMMON_BIALLELIC,
        '-I', bam,
        '-R', GENOME,
        '-O', os.path.join(out_dir, f'{sample}.{char}.allelicCounts.tsv')
    ]
    return ' '.join(pieces)


def model_segments(sample, out_dir):
    pieces = [
        'gatk ModelSegments',
        '--denoised-copy-ratios', os.path.join(out_dir, f'{sample}.T.denoisedCR.tsv'),
        '--allelic-counts', os.path.join(out_dir, f'{sample}.T.allelicCounts.tsv'),
        '--normal-allelic-counts', os.path.join(out_dir, f'{sample}.N.allelicCounts.tsv'),
        '--output', out_dir,
        '--output-prefix', f'{sample}.T'
    ]
    return ' '.join(pieces)


def call_copy_segments(sample, out_dir):
    pieces = [
        'gatk CallCopyRatioSegments',
        '--input', os.path.join(out_dir, f'{sample}.T.cr.seg'),
        '--output', os.path.join(out_dir, f'{sample}.T.called.seg')
    ]
    return ' '.join(pieces)

In [17]:
def plot_denoised(sample, out_dir, min_contig_len=10000000, char='T'):
    pieces = [
        'gatk PlotDenoisedCopyRatios',
        '--standardized-copy-ratios', os.path.join(out_dir, f'{sample}.{char}.standardizedCR.tsv'),
        '--denoised-copy-ratios', os.path.join(out_dir, f'{sample}.{char}.denoisedCR.tsv'),
        '--sequence-dictionary', GENOME_DICT,
        '--minimum-contig-length', str(min_contig_len),
        '--output', os.path.join(out_dir, 'plots'),
        '--output-prefix', f'{sample}.{char}',
        '--tmp-dir', out_dir
    ]
    return ' '.join(pieces)


def plot_segments(sample, out_dir, min_contig_len=10000000, char='T'):
    pieces = [
        'gatk PlotModeledSegments',
        '--denoised-copy-ratios', os.path.join(out_dir, f'{sample}.{char}.denoisedCR.tsv'),
        '--allelic-counts', os.path.join(out_dir, f'{sample}.{char}.hets.tsv'),
        '--segments', os.path.join(out_dir, f'{sample}.{char}.modelFinal.seg'),
        '--sequence-dictionary', GENOME_DICT,
        '--minimum-contig-length', str(min_contig_len),
        '--output', os.path.join(out_dir, 'plots'),
        '--output-prefix', f'{sample}.{char}',
        '--tmp-dir', out_dir
    ]
    return ' '.join(pieces)

In [18]:
# sh $submitJob 1 1 gatk4cn.s5.${caseID} "${PYTHON3} $scriptDir/src/segment_to_geneLevel.py --prefix ${caseID}.T --name ${caseID} --seg ${DIR}/${caseID}.T.called.igv.seg --gene ${ProteinCodingGene} -o ${DIR}"
# sh $submitJob 2 1 gatk4cn.s6.${caseID} "${PYTHON3} $scriptDir/src/mergeMultipleFilesToOne.py ${outdir}"

def gene_level(script, sample, out_dir):
    pieces = [
        'python', script,
        '--prefix', f'{sample}.T',
        '--name', sample,
        '--seg', os.path.join(out_dir, f'{sample}.T.called.igv.seg'),
        '--gene', PROTEIN_CODING_GENE,
        '-o', out_dir
    ]
    return ' '.join(pieces)

def merge_gene_level(script, out_dir):
    pieces = [
        'python', script, out_dir
    ]
    return ' '.join(pieces)

In [20]:
def run_cnv(normal_bam, tumor_bam, sample, out_dir, gene_level_script, merge_gene_script):
    logging.info('collecting tumor reads')
    cmd = collect_read_counts(tumor_bam, sample, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('denoising counts')
    cmd = denoise_counts(sample, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('collect allelic counts normal')
    cmd = collect_allelic_counts(normal_bam, sample, out_dir, 'N')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('collect allelic counts tumor')
    cmd = collect_allelic_counts(tumor_bam, sample, out_dir, 'T')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('modeling segments')
    cmd = model_segments(sample, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('call copy ratio segments')
    cmd = call_copy_segments(sample, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('plot denoised tumor')
    cmd = plot_denoised(sample, out_dir, char='T')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('plot segments tumor')
    cmd = plot_segments(sample, out_dir, char='T')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('plot denoised normal')
    cmd = plot_denoised(sample, out_dir, char='N')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('plot segments normal')
    cmd = plot_segments(sample, out_dir, char='N')
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('running gene level')
    cmd = gene_level(gene_level_script, sample, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
    logging.info('merging gene level')
    cmd = merge_gene_level(merge_gene_script, out_dir)
    logging.info(f'executing command: {cmd}')
    output = subprocess.check_output(cmd, shell=True)
    
